In [1]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [3]:
class CustomHistory(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        
    def on_epoch_end(self, batch, logs={}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))

In [4]:
look_back = 40

# 1. 데이터셋 생성하기
signal_data = np.cos(np.arange(1600)*(20*np.pi/1000))[:,None]

# 데이터 전처리
scaler = MinMaxScaler(feature_range=(0, 1))
signal_data = scaler.fit_transform(signal_data)

# 데이터 분리
train = signal_data[0:800]
val = signal_data[800:1200]
test = signal_data[1200:]

# 데이터셋 생성
x_train, y_train = create_dataset(train, look_back)
x_val, y_val = create_dataset(val, look_back)
x_test, y_test = create_dataset(test, look_back)

# 데이터셋 전처리
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [5]:
# 2. 모델 구성하기
model = Sequential()
model.add(LSTM(32, batch_input_shape=(1, look_back, 1), stateful=True))
model.add(Dropout(0.3))
model.add(Dense(1))

In [6]:
# 3. 모델 학습과정 설정하기
model.compile(loss='mean_squared_error', optimizer='adam')

In [7]:
# 4. 모델 학습시키기
custom_hist = CustomHistory()
custom_hist.init()

In [8]:
for i in range(200):
    model.fit(x_train, y_train, epochs=1, batch_size=1, shuffle=False, callbacks=[custom_hist], validation_data=(x_val, y_val))
    model.reset_states()

Train on 760 samples, validate on 360 samples
Epoch 1/1
760/760 [==============================] - 61s 81ms/step - loss: 0.0423 - val_loss: 0.0094
Train on 760 samples, validate on 360 samples
Epoch 1/1
760/760 [==============================] - 60s 78ms/step - loss: 0.0082 - val_loss: 0.0040
Train on 760 samples, validate on 360 samples
Epoch 1/1
760/760 [==============================] - 67s 88ms/step - loss: 0.0061 - val_loss: 2.3824e-04
Train on 760 samples, validate on 360 samples
Epoch 1/1
760/760 [==============================] - 62s 81ms/step - loss: 0.0046 - val_loss: 3.5072e-04
Train on 760 samples, validate on 360 samples
Epoch 1/1
760/760 [==============================] - 60s 79ms/step - loss: 0.0041 - val_loss: 0.0017
Train on 760 samples, validate on 360 samples
Epoch 1/1
760/760 [==============================] - 60s 79ms/step - loss: 0.0037 - val_loss: 0.0013
Train on 760 samples, validate on 360 samples
Epoch 1/1
760/760 [==============================] - 60s 79ms/st

KeyboardInterrupt: 